<a href="https://colab.research.google.com/github/mobinhs121/OCR/blob/main/OCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!unzip /content/standard-ocr-dataset.zip
!unzip /content/persianocrdataset.zip

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

# Data Preprocessing
def preprocess_data(dataset_path):
    images = []
    labels = []

    for root, dirs, files in os.walk(dataset_path):
        for file in files:
            if file.endswith(".png"):
                image_path = os.path.join(root, file)
                label = os.path.basename(os.path.dirname(image_path))

                # Load and preprocess the image
                image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
                image = cv2.normalize(image, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
                image = cv2.equalizeHist(image)
                image = cv2.GaussianBlur(image, (3, 3), 0)
                images.append(image)
                labels.append(label)

    images = np.array(images)
    labels = np.array(labels)

    return images, labels

# Load and preprocess the English dataset
english_images, english_labels = preprocess_data('path/to/english_dataset')
# Load and preprocess the Persian dataset
persian_images, persian_labels = preprocess_data('path/to/persian_dataset')

# Combine datasets
images = np.concatenate((english_images, persian_images), axis=0)
labels = np.concatenate((english_labels, persian_labels), axis=0)

# Encode labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Model Training
model = models.Sequential([
    layers.Flatten(input_shape=(X_train.shape[1], X_train.shape[2])),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(64, activation='relu'),
    layers.Dense(2, activation='softmax')  # Two classes: English and Persian
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}%")

# Save the model
model.save('text_detection_model.h5')

# Test the model on new images using OpenCV
def test_image(image_path, model):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.normalize(image, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
    image = cv2.equalizeHist(image)
    image = cv2.GaussianBlur(image, (3, 3), 0)
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    prediction = model.predict(image)
    predicted_class = np.argmax(prediction)
    predicted_class = label_encoder.inverse_transform([predicted_class])[0]
    return predicted_class

# Test the model on new images
test_image_path = 'path/to/test_image.png'
predicted_class = test_image(test_image_path, model)
print(f"Predicted Class for {test_image_path}: {predicted_class}")
